### Import library

In [1]:
""" Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP
    Goal is to predict if mutation is SNP or PD
    CV branch
    
    Total samples: 3368
    2254 PD samples
    1111 SNP samples
"""

' Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP\n    Goal is to predict if mutation is SNP or PD\n    CV branch\n    \n    Total samples: 3368\n    2254 PD samples\n    1111 SNP samples\n'

In [2]:
""" Imports the required libraries and packages """

import pandas as pd  # Data manipulation in dataframes
import numpy as np  # Array manipulation

import random as rd # Random seed generation
import time #Time program run time
import sys

import matplotlib.pyplot as plt
from matplotlib.patches import Patch #CV visualise

# import PDB2AC

from sklearn.metrics import(
    matthews_corrcoef,  # MCC for evaluation
    balanced_accuracy_score, #hyperparameter evaluation
    f1_score,  #hyperparameter evaluation
    confusion_matrix,  # confusion matrix for classification evalutation
    classification_report #Return the F1, precision, and recall of a prediction
    )

from sklearn.model_selection import(
    train_test_split,  # Splits data frame into the training set and testing set
    GridSearchCV,  # Searches all hyperparameters
    RandomizedSearchCV, # Searches random range of hyperparameters
    GroupKFold, # K-fold CV with as groups
        )

from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier #SK learn API for classificastion random forests
from sklearn.tree import DecisionTreeClassifier #Single tree decisions 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier #allows for confidence scores to be predicted for each

np.set_printoptions(suppress=True, threshold=np.inf, precision=3) #full array printing

### Split dataset into training and validation sets

In [3]:
def Train_Test_Split(file):
    """      
    Input:      file             Pre-processed dataset done by PDB2AC script

    Returns:    Training_Set     80% training set split
                Testing_Set      20% testing set split
                labels           Class labels for training set

    80% training and 20% testing split. Writes the data to txt files. Splits are shuffled randomly
    """
    AC_dataset = pd.read_csv(file)    
    Training_Set, Testing_Set = train_test_split(AC_dataset,train_size = 0.8)
    labels = Training_Set['dataset_pd'].astype('int32')
    Training_Set.reset_index(drop=True, inplace = True)
    Testing_Set.reset_index(drop=True, inplace = True)
    
#     Training_file = Training_Set.drop(['Binding', 'SProtFT0', 'SProtFT1', 'SProtFT2', 'SProtFT3', 'SProtFT4', 'SProtFT5', 'SProtFT6', 'SProtFT7', 'SProtFT8', 'SProtFT9', 'SProtFT10', 'SProtFT11', 'SProtFT12', 'Interface', 'Relaccess', 'Impact', 'HBonds', 'SPhobic', 'CPhilic', 'BCharge', 'SSGeom', 'Voids', 'MLargest1', 'MLargest2', 'MLargest3', 'MLargest4', 'MLargest5', 'MLargest6', 'MLargest7', 'MLargest8', 'MLargest9', 'MLargest10', 'NLargest1', 'NLargest2', 'NLargest3', 'NLargest4', 'NLargest5', 'NLargest6', 'NLargest7', 'NLargest8', 'NLargest9', 'NLargest10', 'Clash', 'Glycine', 'Proline', 'CisPro'],axis=1)
#     Testing_file = Testing_Set.drop(['Binding', 'SProtFT0', 'SProtFT1', 'SProtFT2', 'SProtFT3', 'SProtFT4', 'SProtFT5', 'SProtFT6', 'SProtFT7', 'SProtFT8', 'SProtFT9', 'SProtFT10', 'SProtFT11', 'SProtFT12', 'Interface', 'Relaccess', 'Impact', 'HBonds', 'SPhobic', 'CPhilic', 'BCharge', 'SSGeom', 'Voids', 'MLargest1', 'MLargest2', 'MLargest3', 'MLargest4', 'MLargest5', 'MLargest6', 'MLargest7', 'MLargest8', 'MLargest9', 'MLargest10', 'NLargest1', 'NLargest2', 'NLargest3', 'NLargest4', 'NLargest5', 'NLargest6', 'NLargest7', 'NLargest8', 'NLargest9', 'NLargest10', 'Clash', 'Glycine', 'Proline', 'CisPro'],axis=1)

    with open('Training set.txt', 'w') as file: #Writes training data to files
        file.write(Training_Set.to_string())
    with open('Testing set.txt', 'w') as file: #Writes testing data to files
        file.write(Testing_Set.to_string())


    return Training_Set, Testing_Set, labels

### Initial evaluation

In [4]:
# def test(Initial_RFC, Input_test, Classes_test):
#     """ Input:  Input_test      Features test data
#                 Classes_test    Class label test data

#         Evaluates the training data before balancing. Random forest classifier makes prediction using the test features. True values 
#         are the class labels testing data
#     """

#     Output_pred = Initial_RFC.predict(Input_test) #Always perdict on the unseen test data, as train has been used by the estimastor
#     print(f"              **Initial Evaluation**\n")
#     print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
#     print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")


## Group K-fold CV (outer loop)

In [5]:
def CV(Training_Set):
    """      
    Input:      Training_Set     80% training set split
            
    Returns:    Input_train     Training features, for each fold
                Classes_train   Ttraining classes, for each fold
                Input_val       Validating features, for each fold
                Classes_val     Validating classes, for each fold

    Group K-fold CV that maintains protein groups, attempts to preserve number of samples of each class 
    for each fold, and ensures protein groups are separated. Creates 5 folds.
    """
    
    Input_CV       = Training_Set.drop(['dataset_pd'], axis =1)         #Input features for training
    Output_CV      = Training_Set['dataset_pd'].copy().astype('int32')  #Class labels for training
    Protein_Groups = Training_Set['AC Code'].to_list()                  #List of proteins for grouping
        
    CV             = GroupKFold(n_splits = 5)                           #Only shuffles proteins in each group, not groups in fold
    
    IT_list = []
    LT_list = []
    IV_list = []
    LV_list = []
    
    for train_idx, val_idx in CV.split(Input_CV, Output_CV, Protein_Groups):
        Rd = np.random.randint(time.time())                             #Random number from 1 to time since epoch

        Input_train                        = Input_CV.loc[train_idx]
        Classes_train                      = Output_CV.loc[train_idx]
        Input_train.drop(['AC Code'], axis = 1, inplace = True)         #Group identifer not needed for training

        Input_val                          = Input_CV.loc[val_idx]
        Classes_val                        = Output_CV.loc[val_idx]
        Input_val.drop(['AC Code'], axis   = 1, inplace = True)
        
        IT_list.append(Input_train.sample(frac=1, random_state=Rd)) 
        LT_list.append(Classes_train.sample(frac=1, random_state=Rd))
        IV_list.append(Input_val.sample(frac=1, random_state=Rd))
        LV_list.append(Classes_val.sample(frac=1, random_state=Rd))
        
    with open('CV validation data.txt', 'w') as f:
        for number, fold in zip(range(len(LV_list)), LV_list):
            f.write(f"Fold: {number}\n\n{fold.to_string()}\n\n\n")

    return(IT_list, LT_list, IV_list, LV_list)


## Balancing (inner loop)

In [6]:
def find_minority_class(classData):
    """ Input:    classData  Array of class labels
    
        Returns:  minClass   The label for the minority class
                  minSize    The number of items in the minority class
                  maxSize    The number of items in the majority class

        Finds information about the inbalance in class sizes
    """
    
    Minority_count = 0
    Majority_count = 0
    for datum in classData:
        if datum == 1:
            Majority_count += 1
        elif datum == 0:
            Minority_count += 1

    minClass = 0
    minSize = Minority_count
    maxSize = Majority_count
    if Minority_count > Majority_count:
        minClass = 1
        minSize = Majority_count
        maxSize = Minority_count

    return minClass, minSize, maxSize

In [7]:
def balance(inData, classData, minClass, minSize):
    """ Input:    inData          array of input data
                  classData       array of classes assigned
                  minorityClass   class label for the minority class
                  minoritySize    size of the minority class

        Returns: array of indexes that are of interest for a balanced dataset

        Perform the actual balancing for a fold between SNPs and PDs
    """
    usedLines = [False] * len(inData) #Array of false for length of data
    for i in range(len(inData)):
        if classData[i] == minClass:
            usedLines[i] = True
            
    usedCount = 0
    while usedCount < minSize:
        i = rd.randrange(len(inData))
        if usedLines[i] == False:
            usedCount += 1
            usedLines[i] = True       

    return usedLines

In [8]:
def balance_data(inData, classData, usedLines):
    """     Input:     inData      array of input training data
                       classData   array of classes assigned to training data
                       usedLines   array of line indexes to print

            Returns:   input_balance  Array of balanced input training data
                       label_balance  Array of balanced labels training data
                       
        Create array of the input training data and classes used. Converts from array to dataframe for CV code compatibility.
        The index [i] is the identifier between the two arrays.
    """
    input_balance = []
    label_balance = []
    
    for i in range(len(inData)):
        if usedLines[i]:
            input_balance.append(inData[i])
            label_balance.append(classData[i])

    input_balance = np.stack(input_balance, axis =0)
    label_balance = np.stack(label_balance, axis =0)     
    
#     Dataframe format
#     input_balance = pd.DataFrame(input_balance, columns = ['Binding', 'SProtFT0', 'SProtFT1', 'SProtFT2', 'SProtFT3', 'SProtFT4', 'SProtFT5', 'SProtFT6', 'SProtFT7', 'SProtFT8', 'SProtFT9', 'SProtFT10', 'SProtFT11', 'SProtFT12', 'Interface', 'Relaccess', 'Impact', 'HBonds', 'SPhobic', 'CPhilic', 'BCharge', 'SSGeom', 'Voids', 'MLargest1', 'MLargest2', 'MLargest3', 'MLargest4', 'MLargest5', 'MLargest6', 'MLargest7', 'MLargest8', 'MLargest9', 'MLargest10', 'NLargest1', 'NLargest2', 'NLargest3', 'NLargest4', 'NLargest5', 'NLargest6', 'NLargest7', 'NLargest8', 'NLargest9', 'NLargest10', 'Clash', 'Glycine', 'Proline', 'CisPro'])
#     label_balance = pd.DataFrame(label_balance, columns = ['Class'])
    
    return input_balance, label_balance

### Balance for n folds

In [9]:
def Balance_ratio(maxSize, minSize): 
    """ Input:      maxSize     The number of items in the majority class
                    minSize     The number of items in the minority class

        Returns:    BF          Number of balancing folds

        Calculate the number of balancing folds needed using ratio of majority to minority class size. Double to ensure sufficient
        majority class instances are sampled, then + 1 to make odd to allow weighted vote.
    """
    Divide = maxSize/minSize
    BF = (2 * round(Divide)) + 1 #Double ratio to nearest integer
    return BF

In [10]:
def Balance_Folds(BF, inData, classData, minClass, minSize):
    """ Input:      BF                Number of balancing folds needed
                    usedLines         Array of line indexes to print
                    input_balance     Dataframe of balanced training data
                    label_balance
                    
        Returns:    Input_folds       List of balanced arrays of training data
                    Output_folds

        Perform the balance_data() function n number of balancing fold times. Return lists for feature data and labels
        where each item is the output of balance_data()
    """
    Input_folds = []
    Output_folds = []
    
    for i in range(BF):
        usedLines = balance(inData, classData, minClass, minSize)
        input_balance, label_balance = balance_data(inData, classData, usedLines)
        
        Input_folds.append(input_balance)
        Output_folds.append(label_balance)
        
#     with open('Balanced training data.txt', 'w') as f:
#         for number, fold in zip(range(BF), train_folds):
#             f.write(f"Fold: {number}\n\n{fold}\n\n\n")
            
    return Input_folds, Output_folds

In [11]:
file = "AC_dataset.csv"
Training_Set, Testing_Set, labels  = Train_Test_Split(file)
IT_list, LT_list, IV_list, LV_list = CV(Training_Set)

for i in range(len(IT_list)): #For every CV fold
    classData                   = LT_list[i].to_numpy()
    inData                      = IT_list[i].to_numpy()
    ValData                     = IV_list[i]
    Vallabel                    = LV_list[i]

    minClass, minSize, maxSize  = find_minority_class(classData)
    
    BF                          = Balance_ratio(maxSize, minSize)
    Input_folds, Output_folds   = Balance_Folds(BF, inData, classData, minClass, minSize) #balance() and balance_data() functions are called under this

#     BF_RFC                      = BF_fitting(Input_folds, Output_folds)
#     Prob_matrix                 = BF_validate(BF_RFC, ValData)

#     Final_vote, Sum_PD, Sum_SNP = Weighted_Vote(Prob_matrix, BF)
#     S_Out                       = Final_score(Sum_PD, Sum_SNP, BF)

#     evalutation(Vallabel, Final_vote, S_Out)

In [32]:
df = pd.DataFrame((Input_folds[2]))
df

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,134.421,127.160,120.386,106.865,105.234,76.646,-1.44,-100.0000,-100.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,106.446,105.943,67.404,52.431,50.834,49.982,-6.17,-100.0000,-100.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,185.892,160.655,141.785,139.007,138.068,108.904,-10.81,-100.0000,-100.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,115.115,103.558,102.872,92.689,86.358,64.521,-7.04,-100.0000,-100.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,173.934,167.457,163.380,162.478,157.641,103.289,-11.52,-100.0000,-100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,188.995,173.359,151.180,140.825,135.452,129.990,-6.99,-100.0000,-100.0,0.0
1410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,104.983,104.133,101.332,99.677,97.654,97.148,31.37,1.7615,-100.0,0.0
1411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,134.062,120.993,115.903,106.835,98.208,96.795,-8.17,-100.0000,-100.0,0.0
1412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,77.774,76.603,64.033,63.727,62.991,56.918,-4.13,-100.0000,-100.0,0.0


### RFC hyperparameter tuning

In [13]:
#  def Hyperparameter(BF, Input_folds, Output_folds):
#     """ Input:      BF                Number of balancing folds needed
#                     Input_folds       List of 5 balanced arrays of training data
#                     Output_folds      List of 5 balanced arrays of training data's labels

#         Returns:    BF_RFC_HP         List of optimized hyperparameters for each RFC

#         Perform RandomSearchCV on each RFC to optimize number of trees, max depth and max samples
#     """  
#     estimator = RandomForestClassifier()
#     param_grid = {
#                 'n_estimators':np.arange(50,500,50),
#                 'max_depth': np.arange(2, 10, 2),
#                 'max_samples': np.arange(0.2, 1.2, 0.2)
#                   }
#     BF_RFC_HP = []

#     for i in range(BF):
#         HPtuning = RandomizedSearchCV(
#             estimator,
#             param_grid, 
#             scoring = 'balanced_accuracy',
#             cv = 10,
#             n_jobs = 6, #how many cores to run in parallel
#             verbose = 2
#             ).fit(Input_folds[i], Output_folds[i].ravel())
#         BF_RFC_HP.append(HPtuning.best_params_)
    
#     return(BF_RFC_HP)

### Train RFC on the trainings folds

In [14]:
def BF_fitting(Input_folds, Output_folds): 
    """ Input:
                    Input_train_list      List of 5 balanced arrays for training data
                    Classes_train_list    List of 5 balanced arrays of training data labels
        Returns:    BF_RFC                List of RFC's trained on data in each balancing fold

        Create RFC model that returns probability predictions for each fold, using output of Balance_Folds() as training data
    """    
    BF_RFC = []
    for i in range(BF):
        BF_RFC.append(RandomForestClassifier(verbose = 1, n_estimators = 1000)) #Generates a RFC for each fold's training data
        BF_RFC[i].fit(Input_folds[i], Output_folds[i]) #Fits the RFC to each folds' training data
        
    return BF_RFC

#### Validate each RFC on validation set, for each fold

In [15]:
def BF_validate(BF_RFC, ValData):
    """ Input:  BF_RFC          List of RFC's trained on data in each balancing fold
                Input_val_list  Unseen validation data fold from CV fold
                
        Returns:Prob_matrix     List of arrays. Each item is 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability
    
        Test the trained RFCs on the test set, then for every instance, outputs the predicted probability for each class
    """
    
    Prob_matrix = [] #Empty list
    for i in range(len(BF_RFC)):
        Prob = BF_RFC[i].predict_proba(ValData.values)
        Prob_matrix.append(Prob)   
            
    with open('Validation fold probabilities.txt', 'w') as f:
        for number, line in zip(range(BF), Prob_matrix ):
            f.write(f"Fold: {number}\n\n   SNP    PD\n{line}\n\n\n")

    return Prob_matrix

### Weighted voting

In [16]:
def Weighted_Vote(Prob_matrix, BF):
    """ Input:      Prob_matrix     List of arrays. 2D matrix where the 1st dimension is each subset in balancing fold, 
                                    2nd dimension is predicted probability
                    BF              Number of balancing folds

        Returns:    Final_vote      Weighted vote classification

        Calculate the final weighted vote using confidence scores (Sc). Binary classification formula Sc = 2|S0 - 0.5|
    """
    Sc_PD = [] #Empty list
    Sc_SNP = [] #Empty list
    for i in range(BF):
        Sc_PD.append(2* (Prob_matrix[i][:,1] - 0.5)) #Confidence scores for PD, for each fold
        Sc_SNP.append(2*(Prob_matrix[i][:,0] - 0.5)) #Confidence scores for SNP, for each fold

    Sum_PD = np.sum(Sc_PD, axis = 0) #Sum of all PD confidence scores. 1D Array
    Sum_SNP = np.sum(Sc_SNP, axis = 0) #Sum of all SNP confidence scores. 1D Array     
    
    Vote_arr = [] #Empty list

    for i in range(len(Sum_PD)):
        if Sum_PD[i] >= Sum_SNP[i]:
            Vote_arr.append([1]) #Append PD classifications to list
        elif Sum_SNP[i] > Sum_PD[i]:
            Vote_arr.append([0]) #Append SNP classifications to list

        Final_vote = np.stack(Vote_arr) #Converts list of arrays to a 2D array
        Final_vote = Final_vote.ravel() #Flattens 2D array to 1D array

    return(Final_vote, Sum_PD, Sum_SNP) #Returns the final confidence scores


### Final confidence

In [17]:
def Final_score(Sum_PD, Sum_SNP, BF):
    """ Input:      Sum_PD      Sum of confidence score for PD predictions
                    Sum_SNP     Sum of confidence score for SNP predictions

        Returns:    S_out       Final confidence score

        Calculate the final confidence score
    """
    
    S_Out = np.abs((Sum_PD - Sum_SNP) /(BF*2))
    np.savetxt('S_out.txt', S_Out, "%.3f")
    
    return S_Out


In [18]:
def evalutation(Vallabel, Final_vote, S_Out):
    """ Input:      Classes_val_list   Validation label data
                    Final_vote         Weighted vote classification

        Evaluation metrics from RFC on validation data with
    """
    Output_pred = Final_vote
    print(f"              ***Final Evaluation***\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Vallabel, Output_pred)}")
    print(f"{classification_report(Vallabel, Output_pred)}\nMCC                {matthews_corrcoef(Vallabel, Output_pred)}")

    print(f"See file 'Classification.txt' for final classifications and confidence scores")
    np.savetxt('Classification.txt',
           np.column_stack([Final_vote, S_Out]),
           fmt = ["%.0f","%.3f"],
           delimiter ="      ",
           header = "Final classifications and confidence scores\n\n"
          )


### Main Program

In [19]:
file = "AC_dataset.csv"
Training_Set, Testing_Set, labels  = Train_Test_Split(file)
IT_list, LT_list, IV_list, LV_list = CV(Training_Set)

for i in range(len(IT_list)): #For every CV fold
    classData                   = LT_list[i].to_numpy()
    inData                      = IT_list[i].to_numpy()
    ValData                     = IV_list[i]
    Vallabel                    = LV_list[i]

    minClass, minSize, maxSize  = find_minority_class(classData)
    BF                          = Balance_ratio(maxSize, minSize)
    Input_folds, Output_folds   = Balance_Folds(BF, inData, classData, minClass, minSize) #balance() and balance_data() functions are called under this

    BF_RFC                      = BF_fitting(Input_folds, Output_folds)
    Prob_matrix                 = BF_validate(BF_RFC, ValData)

    Final_vote, Sum_PD, Sum_SNP = Weighted_Vote(Prob_matrix, BF)
    S_Out                       = Final_score(Sum_PD, Sum_SNP, BF)

    evalutation(Vallabel, Final_vote, S_Out)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 